## 학습목표 
  1. redis db의 이해 및 숙지

* mysql utf-8 설정

* python orm 연습문제)
 1. 나이가 30 이상인 학생들의 평균성적은 얼마인가요?
 2. 나이가 27 이하인 학생들은 어떤 과목을 수강하나요?
 3. 과목별 평균 성적을 구하세요.
 4. min / max 한꺼번에 출력

 * 연습문제) 
  1. world database 에서 각 나라에서 가장 많이 쓰는 언어와 비율을 구하세요
  2. GNP가 가장 높은 나라는?
  3. 각 나라의 주요 도시별 평균 인구수는?
  4. 기대수명이 평균보다 낮은 나라들의 평균 GNP는?
     

* redis
  - in memory key value database
  - REmote DIctionary Server 의 줄임말
  - RDB나 NoSQL이 기본적으로 file system에 데이터를 저장하는데 반해, redis는 메모리에 데이터를 저장한다.
  - 메모리에 데이터를 저장하기 때문에 데이터의 저장/반환이 상대적으로 훨씬 빠르다.
  - 메모리에 데이터를 저장하기 때문에 휘발성의 성격을 갖는다.
  - 주로 cache의 용도로 사용하며 최근 10개의 댓글, 인기 뉴스 등 자주 사용 하는 기능의 cache용도로 사용한다.

* redis 설치
  1. sudo add-apt-repository ppa:chris-lea/redis-server
  2. sudo apt-get update
  3. sudo apt-get install redis-server
  4. sudo vi /etc/redis/redis.conf
  5. #bind 127.0.0.1 로 변경 후 저장
  6. sudo service redis restart
  7. aws 6379 포트 열기
  

* redis 접속
  - 터미널에서 redis-cli 로 접속
  - OSX 설치
    - brew install redis
  - Windows 설치
    - official 하게 windows 지원하지 않음
 
 
* redis python module
  1. sudo pip install redis
  
  
* redis documentation
  - https://redis.io/commands

* Key-value set/get
  - 단순한 key value 값 저장하고 불러오기
  - set key value 
  - e.g) set a "hello"
  - get key
  - e.g) get a
  
  - setex / setnx
    + setex mykey 10 "hello" (10초간 값을 유지)
    + setnx mykey "hello"    (key가 없는 경우에만 설정 nx - not exist)

* Hashes
   - key, value의 집합
   - hset / hget
    - hset myhash field1 "Hello"
    - e.g) hset aaron name "Hello"
    - hget myhash field1
    - e.g) hget aaron name
  
   - hmset / hmget / hgetall
    - hmset myhash field1 "Hello" field2 "World" ...
    - e.g) hmset aaron name "Hello" password "test1234" points 100
    - hmget myhash field1 field2
    - e.g) hmget aaron name password points
    - hmgetall myhash
    - e.g) hmgetall aaron
    
   - hdel 
    - hdel myhash field1
    - e.g) hdel aaron name
    

* List
  - lpush rpush (각각 왼쪽과 오른쪽에서 리스트에 푸쉬함)
    - lpush mylist redis
    - rpush mylist mongodb
  
  - lrange mylist 0 -1 (리스트의 모든 값 출력)
  - lrem mylist count value
    - mylist에서 value와 일치하는 것들을 count 만큼 삭제
    - count > 0 
       + head부터 count만큼 삭제
    - count < 0 
       - tail부터 -count만큼 삭제
    - count == 0 
       - 전체 삭제

* Sets
 - 집합
 - 집합에 추가
   - sadd myset redis
   - sadd myset mongo
 - 집합에서 삭제
   - srem myset redis
 - 멤버인지 확인
   - sismember myset redis 
   - 멤버이면 1, 아니면 0
 - 집합 보기
   - smembers myset

* Key / Value 예제

In [5]:
host = '54.149.163.97'

In [12]:
import redis
r = redis.Redis(host = host, port = 6379)

print r.set('counter', 1)
print r.get('counter')

# 값을 명시한 초 만큼 유지
r.setex('mykey', 10, 5)
print r.get('mykey')

import time
time.sleep(5)

# 5초 후 다시 확인
print r.get('mykey')



True
1
10
None


* hash 예제

In [22]:
# myhash라는 이름의 hash(dictionary)에 aaron(key), value(1) 설정
print r.hset('myhash', 'aaron', 1)

# myhash라는 이름의 hash에 주어진 dict 한꺼번에 설정
print r.hmset('myhash', {'name' : 'aaron', 'password' : 'test1234'})

# myhash에서 name key에 해당 하는 값 출력
print r.hget('myhash', 'name')

# 없는 경우 None 반환
print r.hget('myhash', 'height')

# key list 로 주어진 값의 쌍을 리스트로 반환
print r.hmget('myhash', ['name', 'password'])

# hash 내용 전체 반환 (dictionary)
print r.hgetall('myhash')

# hash에서 명시된 키 삭제
r.hdel('myhash', 'password')

print r.hgetall('myhash')

0
True
aaron
None
['aaron', 'test1234']
{'aaron': '1', 'password': 'test1234', 'name': 'aaron'}
{'aaron': '1', 'name': 'aaron'}


* list 예제

In [36]:
# 키, hash, list 삭제
r.delete('mylist2')

# 왼쪽부터 삽입
r.lpush('mylist2', 23)
r.lpush('mylist2', 22)
r.lpush('mylist2', 27)

# 오른쪽 부터 삽입
r.rpush('mylist2', 21)
r.rpush('mylist2', 29)

print r.lrange('mylist2', 0, -1)

# 왼쪽 제거
r.lpop('mylist2')

print r.lrange('mylist2', 0, -1)

# 오른쪽 제거
r.rpop('mylist2')

print r.lrange('mylist2', 0, -1)

# 길이
print r.llen('mylist2')



['27', '22', '23', '21', '29']
['22', '23', '21', '29']
['22', '23', '21']
3


* set 예제

In [18]:
r.sadd('myset2', 'mysql')
r.sadd('myset2', 'mongo')
r.sadd('myset2', 'redis')

# 전체 멤버 나열
print r.smembers('myset2')

# 멤버인지 아닌지 확인
print r.sismember('myset2', 'redis')
print r.sismember('myset2', 'radis')

set(['redis', 'mongo', 'mysql'])
True
False


* 뉴스 크롤러 예제의 hard-coding된 url redis로 관리하기

In [ ]:
import redis
r = redis.Redis(host = host, port = 6379)

r.lrange('urls', 0, -1)

rpush urls http://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=105
rpush urls http://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=104
rpush urls http://news.naver.com/main/main.nhn?mode=LSD&mid=shm&sid1=103    

실습)
 1. redis에 crawling할 url들을 저장하고, 크롤러가 redis에서 크롤링할 url을 이용하도록 수정해보세요

 * 연습문제)
  1. redis를 이용하여 최근 10개의 기사를 caching 해봅시다
  2. 각 뉴스에 대한 댓글도 모아야 한다는 요구사항이 생겼습니다. 시스템을 설계하고 구현해보세요.
    + 추가로 필요한 table, collection이 있는지 확인
    + mysql, mongo 등 어디에 저장할지 고려
    + module (class) 설계
    + 구현
    + 테스트
    + 배포
   